In [ ]:
pip install pymysql

In [ ]:
import pandas as pd  
pd.options.mode.chained_assignment = None 
import os, json, time, calendar, re
import pyodbc, pymysql    
import mysql.connector
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sqlalchemy import func, create_engine
from config import remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd 

In [ ]:

mydb = mysql.connector.connect(  host=remote_db_endpoint,  user=remote_db_user,  password=remote_db_pwd,  database=remote_db_name) 
mycursor = mydb.cursor() 
sql="""
UPDATE ExpenseCategories 
  SET EC_PatternMapper='' 
  WHERE EC_PatternMapper IS NULL
"""
mycursor.execute(sql) 
mydb.commit() 

In [ ]:
connstr=f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}"
pymysql.install_as_MySQLdb() 

In [ ]:
pymysql.install_as_MySQLdb()
engine = create_engine(connstr) 
conn = engine.connect() 
dfcats = pd.read_sql("SELECT * FROM ExpenseCategories",con=conn)  
conn.close()
dfcats

In [ ]:
engine = create_engine(connstr) 
conn = engine.connect() 
dfcats = pd.read_sql("SELECT EC_ID, EC_Name, EC_PatternMapper FROM ExpenseCategories WHERE EC_PatternMapper IS NOT NULL AND EC_PatternMapper <> '' ",con=conn)  
conn.close()

In [ ]:
def catReplace(c):
    c=str(c) 
    for i, row in dfcats.iterrows(): 
        if re.search(row.EC_PatternMapper.upper(),c.upper()):
            return row.EC_ID, row.EC_Name          
    return 18,'Misc'

In [ ]:
print( catReplace('I AM Shampoo')[0]   )

In [ ]:
def processDataDirectory(dir):
    files = os.listdir(f'Data/{dir}') 
    df = pd.DataFrame()
    for f in files:
        df1 = pd.read_csv(f'Data/{dir}/{f}')
        df = pd.concat([df1, df], axis=0, ignore_index=True)   
    df = df.rename(columns ={
                          'Order Date':'TransDate'
                         ,'Transaction Date':'TransDate'
                         ,'Item Total':'TransAmt'
                         ,'Amount':'TransAmt'
                         ,'Category':'TransCat'
                         ,'Title':'TransDesc' 
                         ,'Description':'TransDesc'})  
    df.TransDate=pd.to_datetime(df['TransDate']).dt.date
    df=df[['TransDate', 'TransDesc', 'TransCat', 'TransAmt']]
    df=df.drop_duplicates(subset=['TransDate','TransDesc','TransAmt'], keep="first")
    df['TransDesc'] = df['TransDesc'].fillna('')
    df['TransAmt'] = df['TransAmt'].apply(lambda x: str(x).replace('$',''))
    df['TransAmt'] = df['TransAmt'].astype('float').abs()     
    df['TransCat'] = df['TransCat'].apply(lambda c: catReplace(c)[1] ) 
    df['TransCatId'] = df['TransCat'].apply(lambda c: catReplace(c)[0] )     
    df['TransKey'] = df['TransDate'].map(lambda x: str(x)) 
    df['TransKey'] = df['TransKey'] +'-'+ df['TransAmt'].map(lambda x: str(x)) 
    df = df.reindex(columns=['TransDate', 'TransDesc', 'TransCat','TransAmt','TransKey'])  
    df.to_csv(f'Data/df_{dir}.csv', index=False) 
    df.to_json(f'Data/df_{dir}.json', orient='records')
    return df


In [ ]:
dirs = [f.path.replace('\\','/').replace('Data/','') for f in os.scandir('Data') if f.is_dir()]
df_tmas = pd.DataFrame();
for d in dirs:
    df = processDataDirectory(d)
    df_tmas = pd.concat([df_tmas, df], axis=0, ignore_index=True) 
df_tmas

In [ ]:

def terser(conetnt): 
    words=msg.split(' ') 
    stop_words = set(stopwords.words('english')) 
    filtered = []
    stemmer = PorterStemmer() 
    for word in words: 
        if word not in stop_words and word.isalpha(): 
            stemmed_word = stemmer.stem(word) 
            filtered.append(stemmed_word)
    return '-'.join(filtered)
msg = '''All work and not play makes jack a dull boy. To be or not to be.
nobody expects the spanish inquisition'''
print(terser(msg))
